In [1]:
import pandas as pd
import glob
import os
from scipy.interpolate import Akima1DInterpolator
from scipy.signal import normalize
from matplotlib import pyplot as plt 
import numpy as np


path = r"./processed_data/charge/"

def get_DTEV(volatge,v_t,vs,ve,cv_p):
    t = np.where((max(vs,min(volatge[0:cv_p]))<volatge[0:cv_p]) & (volatge[0:cv_p]<min(np.max(volatge[0:cv_p]),ve)))[0]
    # print(t)
    return v_t[t.max()]-v_t[t.min()]

def get_CCCT(i_s):
    cc_range = 0.97
    times = 20
    slope = 1.0041
    ccc_p = np.array([])

    while ccc_p.size==0:
        times+=1
        cc = np.where(i_s-i_s.max()*cc_range>0)[0]
        condition = np.where(np.diff(i_s)[cc]-slope<0)[0]
        c= cc.min()+condition
        ccc_p = np.append(ccc_p,c).astype(np.int32) 
        for p in ccc_p:
            if(np.average(i_s[np.max([1,p-100]):p])/np.average(i_s[p:np.min([i_s.size,p+100])])>slope):
                ccc_p = p
                break
            elif p==ccc_p[-1]:
                ccc_p = ccc_p.min()

        slope-=0.0001
        cc_range-=0.001
        if times>100:
            ccc_p=0
            return 1
    return ccc_p

def get_CVCT(v):
    cv_range = 0.997
    times = 20
    slope = 1.00007
    ccv_p = np.array([])
    v_s = v[0:int(v.size*0.8)]

    while ccv_p.size==0:
        times+=1
        cv = np.where(v_s-v_s.max()*cv_range>0)[0]
        # print(cv)
        condition = np.where(np.diff(v_s)[cv-1]<0)[0]
        c= cv.min()+condition
        ccv_p = np.append(ccv_p,c-2).astype(np.int32) 
        for p in ccv_p:
            if(np.average(v_s[p:np.min([v_s.size,p+50])]/np.average(v_s[np.max([1,p-50]):p]))>slope):
                ccv_p = p
                break
            elif p==ccv_p[-1]:
                ccv_p = ccv_p.min()
        slope-=0.00001
        cv_range-=0.0001
        if times>100:
            ccv_p=0
            return 1
    return ccv_p

def get_CVDT(d_current):
    return 0

def get_IC_peaks(IC,voltage):
    vol = voltage
    IC_d = np.diff(IC)
    IC_d2 = np.diff(IC_d)
    v_d = np.diff(vol)
    v_d2 = np.diff(v_d)
    # k1 = np.diff(IC)
    # print(k1)
    k = -(v_d[0:-1]*IC_d2-v_d2*IC_d[0:-1])/np.power(np.power(v_d[0:-1],2)+np.power(IC_d[0:-1],2),1.5)

    k[k<0] = 0
    nor_k = -(k-np.min(k))/(np.max(k)-np.min(k))
    
    nor_IC = -((IC-np.min(IC))/(np.max(IC)-np.min(IC)))[0:-2]

    K = np.multiply(nor_k,nor_IC)

    return K

def fit_func(x,a1,b1,c1,a2,b2,c2,a3,b3,c3,a4,b4,c4):
    return a1*np.sin(b1*x+c1)+a2*np.sin(b2*x+c2)+a3*np.sin(b3*x+c3)+a4*np.sin(b4*x+c4)

def fit_func2(x,*args):
    #拋物線擬和
    val=0
    # print(args)
    dim = len(args)/3
    for i in range(0,int(dim)):
        # print(arg)
        val+=args[0+i*3]*np.sin(args[1+i*3]*x+args[2+i*3])

    return val


In [2]:
from scipy.interpolate import Akima1DInterpolator
from matplotlib import pyplot as plt
import plotly.express as px
from EntropyHub import SampEn

import plotly.io as pio
import plotly.graph_objects as go
from scipy.signal import savgol_filter
from scipy import optimize as op
from scipy.signal import find_peaks,peak_widths
import re
import warnings
warnings.filterwarnings("ignore")

pio.renderers.default = "browser"
plotly_config = dict({"scrollZoom": True,'modeBarButtonsToAdd':[
                                        'drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})



battery_list = os.listdir(path)[-10:]
# battery_list = ["B0056"]


# del battery_list[0:-8]

for battery in battery_list:   
    i=0
    # file_list = glob.glob(path+battery+"/"+f"{battery}_charge_cycle_*")
    file_list = [f"{path}{battery}/{battery}_charge_cycle_{i}.csv" for i in range(1,len(glob.glob(path+battery+"/"+f"{battery}_charge_cycle_*"))+1)]
    cap_list = f"{path}{battery}/{battery}_capacity.csv"
    cap = pd.read_csv(cap_list)
    cycle = 1
    loop=0
    ccc_arr = np.array([0])
    ccv_arr = np.array([0])
    v37_419_arr = np.array([0])
    v38_419_arr = np.array([0])
    v37_41_arr = np.array([0])
    v38_41_arr = np.array([0])
    peak_v_arr1 = np.array([0])
    peak_v_arr2 = np.array([0])
    peak_v_arr3 = np.array([0])
    peak_IC_arr1 = np.array([0])
    peak_IC_arr2 = np.array([0])
    peak_IC_arr3 = np.array([0])
    tem_arr = np.array([0])
    peak_v_arr = np.array([0])
    peak_IC_arr = np.array([0])
    cap_arr = np.array([0])
    for file in file_list[1:]:

        
        df = pd.read_csv(file)
        pattern = re.compile(r'(?<=\D_)[0-9]+(?=\D)')
        w = re.findall(pattern, file)
        # w1 = pattern.match(file)
        # w2 = w1.group()
        loop = int(w[0])
        try:
            cap_loop = cap.loc[loop-1,"Capacity"]
        except:
            pass
        df["Voltage_measured"] = df["Voltage_measured"].astype(np.float64).fillna(method='ffill')
        df["Current_measured"] = df["Current_measured"].astype(np.float64).fillna(method='ffill')
        df["Time"] = df["Time"].astype(np.float64)
        
        try:
            if df["Voltage_measured"].size>201:
                vol = df["Voltage_measured"].to_numpy()
                t = df["Time"].to_numpy()
                cur = df["Current_measured"].to_numpy()
                Aki_vol = Akima1DInterpolator(t,vol)
                Aki_cur = Akima1DInterpolator(t,cur)
                A_t = np.array(range(0,int(t.max())*10))/10
                v = Aki_vol(A_t)
                i = Aki_cur(A_t)

                order = 1
                fremlen = 501
                i_s = savgol_filter(i,fremlen,order)
                v_s = savgol_filter(v,fremlen,order)

                if np.where(v_s>=4.2)[0].size == 0:
                    v42 = len(v_s)-1
                else:
                    v42 = np.where(v_s>=4.2)[0][0]
                # print(v42[0])
                IC_range = [3.7,4.185]
                p = np.where((v_s[0:v42+1]>=IC_range[0]) & (v_s[0:v42+1]<=IC_range[1]))[0]

                # if np.diff(p).any()<0:
                    

                peak_range = [3.75+0.0005*loop,4.175+0.0005*loop]
                p1 = np.where((v_s[0:v42+1]>=max(IC_range[0],peak_range[0])) & (v_s[0:v42+1]<=min(IC_range[1],peak_range[1])))[0]
                if p1.size <100:
                    continue
                vol42 = v_s[p1]
                IC_org = (i_s[p]/3600)[0:-1]/np.diff(v_s[p])

                IC = (i_s[p1]/3600)[0:-1]/np.diff(vol42)
                vol42 = vol42[0:-1]
                order2 = 1
                fremlen2 = 1001
                if fremlen2>len(IC):
                    fremlen2 = int(len(IC)/2)*2-1
                IC_s = savgol_filter(IC,fremlen2,order2)

                v_s_p = v_s[p][0:-1]
                is_sorted = lambda a: np.all(a[:-1] < a[1:])
                if is_sorted(v_s_p)==False:
                    v_s_p_i = np.argsort(v_s_p)
                    v_s_p = v_s_p[v_s_p_i]
                    IC_org = IC_org[v_s_p_i]

                Aki_IC = Akima1DInterpolator(v_s_p,IC_org)
                IC_s2 = Aki_IC(vol42)
                IC_s3 = savgol_filter(IC_s2,int(len(vol42)/2)*2-1,20)

                n=1
                IQR = np.percentile(IC_s3,100) - np.percentile(IC_s3,0)
                IC_s3_1=IC_s3[IC_s3 < np.percentile(IC_s3,75)+n*IQR ]
                IC_s3_1=IC_s3[IC_s3 > np.percentile(IC_s3,25)-n*IQR]
                out_p = np.where(np.append(IC_s3 < np.percentile(IC_s3,25)-n*IQR,IC_s3 > np.percentile(IC_s3,75)+n*IQR))[0]
                # p1 = np.delete(p1,out_p)
                # vol42 = np.delete(vol42,out_p)
                IC_s3_1 = np.delete(IC_s3,out_p)
                IC_s3_1_1 = np.delete(IC_s3_1,np.where(IC_s3_1<0.01))
                p1 = np.delete(p1,out_p)
                p1 = np.delete(p1,np.where(IC_s3_1<0.01))
                vol42 = np.delete(vol42,out_p)
                vol42 = np.delete(vol42,np.where(IC_s3_1<0.01))
                
                x = vol42
                y = IC_s3_1_1
                k = get_IC_peaks(y,x)
                # y = fit_func2(x,*args)

                vol_p = np.where((vol42[0:-1]>=peak_range[0]) & (vol42[0:-1]<=peak_range[1]))[0]
                vol_p1 = vol_p[0]
                max_k_p = vol_p1+np.argmax(k)
                peaks, _ = find_peaks(k,prominence=0.03)
                v_peaks1 = x[peaks]
                v_peaks = v_peaks1[v_peaks1<peak_range[0]+(peak_range[1]-peak_range[0])*0.98]
                IC_peaks1 = y[peaks]
                IC_peaks = IC_peaks1[v_peaks1<peak_range[0]+(peak_range[1]-peak_range[0])*0.98]


                # peak_range2 = [3.77+0.0006*loop,3.87+0.0006*loop]
                # p1_1 = np.where((v_s[0:v42+1]>=peak_range2[0]) & (v_s[0:v42+1]<=peak_range2[1]))[0]
                # vol42_1 = v_s[p1_1]
                # IC_s2_2 = Aki_IC(vol42_1)
                # IC_s3_2 = savgol_filter(IC_s2_2,int(len(vol42_1)/2)*2-1,6)

                # n1=1.5
                # IQR_range = [75,25]
                # IQR_1 = np.percentile(IC_s3_2,IQR_range[0]) - np.percentile(IC_s3_2,IQR_range[1])
                # IC_s3_3=IC_s3_2[IC_s3_2 < np.percentile(IC_s3_2,IQR_range[0])+n1*IQR_1]
                # IC_s3_3=IC_s3_2[IC_s3_2 > np.percentile(IC_s3_2,IQR_range[1])-n1*IQR_1]
                # out_p_1 = np.where(np.logical_or(IC_s3_2 < np.percentile(IC_s3_2,IQR_range[1])
                #                                 -n1*IQR_1,IC_s3_2 > np.percentile(IC_s3_2,IQR_range[0])+n1*IQR_1))[0]
                # IC_s3_3 = np.delete(IC_s3_2,out_p_1)
                # IC_s3_3_2 = np.delete(IC_s3_3,np.where(IC_s3_3<0.01))
                # p1_1 = np.delete(p1_1,out_p_1)
                # p1_1 = np.delete(p1_1,np.where(IC_s3_3<0.01))
                # vol42_1 = np.delete(vol42_1,out_p_1)
                # vol42_1 = np.delete(vol42_1,np.where(IC_s3_3<0.01))
                
                # x1 = vol42_1
                # y1 = IC_s3_3_2
                # k1 = get_IC_peaks(y1,x1)
                # k1 = k1[0:int(0.95*len(k1))]
                # x1 = x1[0:int(0.95*len(x1))]
                # y1 = y1[0:int(0.95*len(y1))]
                # # y = fit_func2(x,*args)

                # vol_p_1 = np.where((vol42_1[0:-1]>=peak_range2[0]) & (vol42_1[0:-1]<=peak_range2[1]))[0]
                # vol_p1_1 = vol_p_1[0]
                # max_k_p_1 = vol_p1_1+np.argmax(k1)
                # peaks1, _ = find_peaks(k1,prominence=0.2)
                # if(peaks1.size != 0):
                #     peaks1 = peaks1[0]
                #     v_peaks1 = np.array([x1[peaks1]])
                #     IC_peaks1 = np.array([y1[peaks1]])
                # else:
                #     v_peaks1 = np.array([])
                    # IC_peaks1 = np.array([])
                    
                # v_peaks =  np.concatenate([v_peaks1,v_peaks])
                # IC_peaks = np.concatenate([IC_peaks1,IC_peaks])





                # dim1 = 2
                # args1 =  op.curve_fit(fit_func2,vol42[p1_1],IC_s[p1_1],
                #                         np.ones(dim1*3),
                #                         maxfev =5000000)[0]

                # x1 = v_s[p1_1]
                # y1 = fit_func2(x1,*args1)
                # # print(args1)
                # vol_p2 = np.where((vol42[0:-1]>=peak_range2[0]) & (vol42[0:-1]<=peak_range2[1]))[0]
                # vol_p3 = vol_p2[0]
                # k1 = get_IC_peaks(y1,x1)
                # max_k_p1 = vol_p1+np.argmax(k1)
                # peaks1, _ = find_peaks(k1,height=0.85)
                # v_peaks1 = x1[peaks1]
                # IC_peaks1 = y1[peaks1]



                ccc_p = get_CCCT(i_s)
                ccc_t = A_t[ccc_p]
                ccv_p = get_CVCT(v_s)
                ccv_t = A_t.max()-A_t[ccv_p]
                v37_419 = get_DTEV(v_s,A_t,3.7,4.19,ccv_p)
                v38_419 = get_DTEV(v_s,A_t,3.8,4.19,ccv_p)
                v37_41 = get_DTEV(v_s,A_t,3.7,4.1,ccv_p)
                v38_41 = get_DTEV(v_s,A_t,3.8,4.1,ccv_p)

                # print(k)

                ccc_arr = np.append(ccc_arr,ccc_t)
                ccv_arr = np.append(ccv_arr,ccv_t)
                v37_419_arr = np.append(v37_419_arr,v37_419)
                v38_419_arr = np.append(v38_419_arr,v38_419)
                v37_41_arr = np.append(v37_41_arr,v37_41)
                v38_41_arr = np.append(v38_41_arr,v38_41)
                tem_arr = np.append(tem_arr,df["Temperature_measured"].mean())
                if v_peaks.size != 0:
                    peak_v_arr1 = np.append(peak_v_arr1,v_peaks[0])
                    peak_v_arr2 = np.append(peak_v_arr2,v_peaks[int(len(v_peaks)/2)])
                    peak_v_arr3 = np.append(peak_v_arr3,v_peaks[-1])
                    peak_IC_arr1 = np.append(peak_IC_arr1,IC_peaks[0])
                    peak_IC_arr2 = np.append(peak_IC_arr2,IC_peaks[int(len(IC_peaks)/2)])
                    peak_IC_arr3 = np.append(peak_IC_arr3,IC_peaks[-1])
                else:
                    peak_v_arr1 = np.append(peak_v_arr1,peak_v_arr1[-1])
                    peak_v_arr2 = np.append(peak_v_arr2,peak_v_arr2[-1])
                    peak_v_arr3 = np.append(peak_v_arr3,peak_v_arr3[-1])
                    peak_IC_arr1 = np.append(peak_IC_arr1,peak_IC_arr1[-1])
                    peak_IC_arr2 = np.append(peak_IC_arr2,peak_IC_arr2[-1])
                    peak_IC_arr3 = np.append(peak_IC_arr3,peak_IC_arr3[-1])

                peak_v_arr = np.append(peak_v_arr,v_peaks[0:3])
                peak_IC_arr = np.append(peak_IC_arr,IC_peaks[0:3])
                cap_arr = np.append(cap_arr,cap_loop)
                

                #plot current
                if(loop%50==2):
                    # fig = px.scatter(df,x="Time",y="Current_measured")
                    # fig.update_layout(
                    #     dragmode='drawopenpath',
                    #     newshape_line_color='cyan',
                    #     title_text=f'{battery}_Current_measured_{loop}', 
                    # )
                    # fig.add_scatter(x=A_t,y=i,name="origin data(Amakima)")
                    # fig.add_scatter(x=A_t,y=i_s,name='savgol_filter')
                    # fig.add_shape(type="circle",
                    #     xref="x", yref="y",
                    #     x0=A_t[ccc_p]-12, y0=i_s[ccc_p]-0.015, x1=A_t[ccc_p]+12, y1=i_s[ccc_p]+0.015,
                    #     line_color="LightSeaGreen",
                    #     name="CCCT"
                    # )
                    # fig.show(config=plotly_config)

                    #plot voltage
                    # fig2 = px.scatter(df,x="Time",y="Voltage_measured")
                    # fig2.update_layout(
                    #     dragmode='drawopenpath',
                    #     newshape_line_color='cyan',
                    #     title_text=f'{battery}_Voltage_measured_{loop}'
                    # )
                    # fig2.add_scatter(x=A_t,y=v,name="origin data(Amakima)")
                    # fig2.add_scatter(x=A_t,y=v_s,name='savgol_filter')
                    # fig2.add_shape(type="circle",
                    #     xref="x", yref="y",
                    #     x0=A_t[ccv_p]-32, y0=v_s[ccv_p]-0.01, x1=A_t[ccv_p]+32, y1=v_s[ccv_p]+0.01,
                    #     line_color="LightSeaGreen",
                    #     name="CVCT"
                    # )
                    # fig2.show(config=plotly_config)

                    fig3 = px.line(x=v_s_p,y=IC_org)
                    fig3.update_layout(
                        dragmode='drawopenpath',
                        newshape_line_color='cyan',
                        title_text=f'{battery}_IC_{loop}', 
                    )
                    fig3.add_trace(
                            go.Scatter(
                                mode='markers',
                                x=v_peaks,
                                y=IC_peaks,
                                marker=dict(
                                    color='rgba(250, 100, 100, 0.5)',
                                    size=20,
                                    line=dict(
                                        color='MediumPurple',
                                        width=2
                                    )
                                ),
                                showlegend=False
                            )
                        )
                    # fig3.add_shape(type="circle",
                    #     xref="x", yref="y",
                    #     x0=vol42[max_k_p]-0.01, y0=IC_s[max_k_p]-0.1, x1=vol42[max_k_p]+0.01, y1=IC_s[max_k_p]+0.01,
                    #     line_color="LightSeaGreen",
                    #     name="PEAK"
                    # )
                    fig3.add_scatter(x=x[0:-2] ,y=k*40,name='K')
                    fig3.add_scatter(x=x ,y=y,name='fit')

                    # fig3.add_scatter(x=x1 ,y=k1*40,name='K1')
                    # fig3.add_scatter(x=x1 ,y=y1,name='fit1')
                    fig3.show(config=plotly_config)

                    # fig4 = px.line(x=vol42[0:-1][vol_p[0:-2]] ,y=k)
                    # fig4.show(config=plotly_config)

            else:
                ccc_arr = np.append(ccc_arr,ccc_arr[-1])
                ccv_arr = np.append(ccv_arr,ccv_arr[-1])
                v37_419_arr = np.append(v37_419_arr,v37_419_arr[-1])
                v38_419_arr = np.append(v38_419_arr,v38_419_arr[-1])
                v37_41_arr = np.append(v37_41_arr,v37_41_arr[-1])
                v38_41_arr = np.append(v38_41_arr,v38_41_arr[-1]) 
                peak_v_arr1 = np.append(peak_v_arr1,peak_v_arr1[-1])
                peak_v_arr2 = np.append(peak_v_arr2,peak_v_arr2[-1])
                peak_v_arr3 = np.append(peak_v_arr3,peak_v_arr3[-1])
                peak_IC_arr1 = np.append(peak_IC_arr1,peak_IC_arr1[-1])
                peak_IC_arr2 = np.append(peak_IC_arr2,peak_IC_arr2[-1])
                peak_IC_arr3 = np.append(peak_IC_arr3,peak_IC_arr3[-1])
                peak_v_arr = np.append(peak_v_arr,peak_v_arr[-1])
                peak_IC_arr = np.append(peak_IC_arr,peak_IC_arr[-1])
                tem_arr = np.append(tem_arr,tem_arr[-1])
                cap_arr = np.append(cap_arr,cap_arr[-1])
        except Exception as e:
            ccc_arr = np.append(ccc_arr,ccc_arr[-1])
            ccv_arr = np.append(ccv_arr,ccv_arr[-1])
            v37_419_arr = np.append(v37_419_arr,v37_419_arr[-1])
            v38_419_arr = np.append(v38_419_arr,v38_419_arr[-1])
            v37_41_arr = np.append(v37_41_arr,v37_41_arr[-1])
            v38_41_arr = np.append(v38_41_arr,v38_41_arr[-1]) 
            peak_v_arr1 = np.append(peak_v_arr1,peak_v_arr1[-1])
            peak_v_arr2 = np.append(peak_v_arr2,peak_v_arr2[-1])
            peak_v_arr3 = np.append(peak_v_arr3,peak_v_arr3[-1])
            peak_IC_arr1 = np.append(peak_IC_arr1,peak_IC_arr1[-1])
            peak_IC_arr2 = np.append(peak_IC_arr2,peak_IC_arr2[-1])
            peak_IC_arr3 = np.append(peak_IC_arr3,peak_IC_arr3[-1])
            peak_v_arr = np.append(peak_v_arr,peak_v_arr[-1])
            peak_IC_arr = np.append(peak_IC_arr,peak_IC_arr[-1])
            tem_arr = np.append(tem_arr,tem_arr[-1])
            cap_arr = np.append(cap_arr,cap_arr[-1])
        # print(str(e))
        # print(battery)
        print(loop)
            
            
    loop+=1
        
    
    features = pd.DataFrame(columns=["CCCT","CVCT","V37_419","V38_419","V37_41","V38_41"])
    features["CCCT"] = ccc_arr
    features["CVCT"] = ccv_arr
    features["V37_419"] = v37_419_arr
    features["V38_419"] = v38_419_arr
    features["V37_41"] = v37_41_arr
    features["V38_41"] = v38_41_arr
    features["Temperature"] = tem_arr
    
    features["peak_v_arr1"] = peak_v_arr1
    features["peak_IC_arr1"] = peak_IC_arr1
    features["peak_v_arr2"] = peak_v_arr2
    features["peak_IC_arr2"] = peak_IC_arr2
    features["peak_v_arr3"] = peak_v_arr3
    features["peak_IC_arr3"] = peak_IC_arr3
    features["Capacity"] = cap_arr
    
    path2 = "./features/charge/"
    features.to_csv(path2+battery+"/"+f"{battery}_Features.csv")



2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
18
19
20
21
22
23
24
25
2
3
4
6
7
8
9
10
11
12
14
15
16
17
20
21
22
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
2
3
4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91

In [ ]:

fig = px.scatter(df,x="Time",y="Current_measured")
fig.update_layout(
    dragmode='drawopenpath',
    newshape_line_color='cyan',
    title_text=f'{battery}_Current_measured_{loop}'
)
fig.add_scatter(x=A_t,y=v)
fig.add_scatter(x=A_t,y=v_s)
# fig.add_shape(type="circle",
#     xref="x", yref="y",
#     x0=A_t[ccc_p]-12, y0=i_s[ccc_p]-0.015, x1=A_t[ccc_p]+12, y1=i_s[ccc_p]+0.015,
#     line_color="LightSeaGreen",
# )
fig.show(config=plotly_config)

: 

In [ ]:
import pandas as pd
import glob
import os
from scipy.interpolate import Akima1DInterpolator
from matplotlib import pyplot as plt 
import numpy as np
from scipy.interpolate import Akima1DInterpolator
from matplotlib import pyplot as plt
import plotly.express as px
from EntropyHub import SampEn
import plotly.io as pio
import plotly.graph_objects as go
from scipy.signal import savgol_filter
path = r"./processed_data/charge/"
battery_list = os.listdir(path)
pio.renderers.default = "browser"
# del battery_list[0:-8]
plotly_config = dict({"scrollZoom": True,'modeBarButtonsToAdd':[
                                        'drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})
fig = go.Figure()
for battery in ["B0005"]:   
    # file_list = glob.glob(path+battery+"/"+f"{battery}_charge_cycle_*")
    file_list = [f"{path}{battery}/{battery}_charge_cycle_{i}.csv" for i in range(1,169)]
    fea = pd.read_csv(path+battery+"/"+f"{battery}_Features.csv")
    tem_arr = np.array([])
    loop=0
    for file in file_list:
        print(file)
        loop+=1
        df = pd.read_csv(file)
        df["Voltage_measured"] = df["Voltage_measured"].astype(np.float64)
        fig.add_trace(go.Scatter(
            y=df["Voltage_measured"],
            name=f'Voltage {file}',
            mode='lines',
            marker_color=f'rgba({loop}, {loop}, 255, .8)'
        ))
        # fig = fig.add_scatter(y=df["Voltage_measured"],fillcolor=)
        fig.update_layout(
            dragmode='drawopenpath',
            newshape_line_color='cyan',
            title_text=f'{battery}_Current_measured_{loop}'
        )
fig.show(config=plotly_config)
    # fea["Temperature"] = tem_arr
    # fea.to_csv(path+battery+"/"+f"{battery}_Features.csv")

: 

In [ ]:
import pandas as pd
import shutil
meta = pd.read_csv("metadata.csv")
chargedata = meta.loc[meta["type"]=="charge"]
dischargedata = meta.loc[meta["type"]=="discharge"]
impedancedata = meta.loc[meta["type"]=="impedance"]
battery_id = chargedata.groupby("battery_id")
# df2 = pd.DataFrame([])
fig = go.Figure()
for bg in ["B0005"]:
    battery = battery_id.get_group(bg)
    cycle = 0
    file = meta[meta["battery_id"]==bg]
    for csvname in battery["filename"]:
        it = meta[meta["filename"]==csvname].index
        if it-1>min(file.index):
            nx = file.loc[it-1,["type"]]
            if (nx["type"] == "charge").bool():
                print(file.loc[it,["filename"]])
                continue
        # df2 = pd.concat(df2,pd.read_csv(f"data/{csvname}")["Voltage_measured"],axis=1)
        if not os.path.isdir(f"processed_data/charge/{bg}"):
            os.makedirs(f"processed_data/charge/{bg}")
        cycle+=1
        df_tmp = pd.read_csv(f"data/{csvname}")
        fig.add_trace(go.Scatter(
            y=df_tmp["Voltage_measured"],
            name=f'Voltage {cycle}',
            mode='lines',
            marker_color=f'rgba({cycle}, {cycle}, 255, .8)'
        ))
        # fig = fig.add_scatter(y=df["Voltage_measured"],fillcolor=)
        fig.update_layout(
            dragmode='drawopenpath',
            newshape_line_color='cyan',
            title_text=f'{battery}_Current_measured_{cycle}'
        )
        
        # cycle+=1
        df_tmp.to_csv("processed_data/charge/{}/{}_charge_cycle_{}.csv".format(bg,bg,cycle))
fig.show(config=plotly_config)
    

: 

In [ ]:
peak_v_arr1 = np.append(peak_v_arr1,0)
peak_v_arr2 = np.append(peak_v_arr2,0)
peak_v_arr3 = np.append(peak_v_arr3,0)
peak_IC_arr1 = np.append(peak_IC_arr1,0)
peak_IC_arr2 = np.append(peak_IC_arr2,0)
peak_IC_arr3 = np.append(peak_IC_arr3,0)

In [ ]:
features["CCCT"] = ccc_arr
features["CVCT"] = ccv_arr
features["V37_419"] = v37_419_arr
features["V38_419"] = v38_419_arr
features["V37_41"] = v37_41_arr
features["V38_41"] = v38_41_arr
features["Temperature"] = tem_arr

features["peak_v_arr1"] = peak_v_arr1
features["peak_IC_arr1"] = peak_IC_arr1
features["peak_v_arr2"] = peak_v_arr2
features["peak_IC_arr2"] = peak_IC_arr2
features["peak_v_arr3"] = peak_v_arr3
features["peak_IC_arr3"] = peak_IC_arr3

print(r"./features/charge/"+battery+"/"+f"{battery}_Features.csv")
features.to_csv("./features/charge/"+battery+"/"+f"{battery}_Features.csv")

./features/charge/B0005/B0005_Features.csv


In [7]:
import pandas as pd
import plotly.express as px
from scipy.stats import pearsonr

plotly_config = dict({"scrollZoom": True,'modeBarButtonsToAdd':[
                                        'drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})
battery = "B0046"
features = pd.read_csv(f"./features/charge/{battery}/{battery}_Features.csv")[1:-2]
SOH = pd.read_csv(f"./features/charge/{battery}/{battery}_Features.csv")["Capacity"][1:-2]

peak3_pear = pearsonr(SOH,features["peak_v_arr3"])
peak3_IC_pear = pearsonr(SOH,features["peak_IC_arr3"])
peak2_pear = pearsonr(SOH,features["peak_v_arr2"])
peak2_IC_pear = pearsonr(SOH,features["peak_IC_arr2"])
peak1_pear = pearsonr(SOH,features["peak_v_arr1"])
peak1_IC_pear = pearsonr(SOH,features["peak_IC_arr1"])
CCCT_pear = pearsonr(SOH,features["CCCT"])
CVCT_pear = pearsonr(SOH,features["CVCT"])
V37_419_pear = pearsonr(SOH,features["V37_419"])
V38_419_pear = pearsonr(SOH,features["V38_419"])
V37_41_pear = pearsonr(SOH,features["V37_41"])
V38_41_pear = pearsonr(SOH,features["V38_41"])
Temperature_pear = pearsonr(SOH,features["Temperature"])
# peak3_pear = pearsonr(SOH,features["V38_419"])
print("peak3_pear: ",peak3_pear)
print("peak3_IC_pear: ",peak3_IC_pear)
print("peak2_pear: ",peak2_pear)
print("peak2_IC_pear: ",peak2_IC_pear)
print("peak1_pear: ",peak1_pear)
print("peak1_IC_pear: ",peak1_IC_pear)
print("CCCT_pear: ",CCCT_pear)
print("CVCT_pear: ",CVCT_pear)
print("V37_419_pear: ",V37_419_pear)
print("V38_419_pear: ",V38_419_pear)
print("V37_41_pear: ",V37_41_pear)
print("V38_41_pear: ",V38_41_pear)
print("Temperature_pear: ",Temperature_pear)
# print("peak3_pear: ",peak3_pear)

fig5 = px.scatter(features,y=["CCCT","CVCT","V37_419","V38_419","V37_41","V38_41","Temperature"])
fig5.add_scatter(y=SOH*2000,name="SOH")
fig5.add_scatter(y = features["peak_v_arr1"]*1000,name="peak_v_arr1")
fig5.add_scatter(y = features["peak_v_arr2"]*1000,name="peak_v_arr2")
fig5.add_scatter(y = features["peak_v_arr3"]*1000,name="peak_v_arr3")
fig5.add_scatter(y = features["peak_IC_arr1"]*100,name="peak_IC_arr1")
fig5.add_scatter(y = features["peak_IC_arr2"]*100,name="peak_IC_arr2")
fig5.add_scatter(y = features["peak_IC_arr3"]*100,name="peak_IC_arr3")
fig5.show(config=plotly_config)

fig6 = px.scatter(features,y=["peak_IC_arr1","peak_IC_arr2","peak_IC_arr3"])
fig6.show(config=plotly_config)

peak3_pear:  PearsonRResult(statistic=0.19132144810509327, pvalue=0.11530556134705838)
peak3_IC_pear:  PearsonRResult(statistic=0.503168691477666, pvalue=1.0504561983409092e-05)
peak2_pear:  PearsonRResult(statistic=0.26417043279992347, pvalue=0.02827853386532194)
peak2_IC_pear:  PearsonRResult(statistic=0.4467973996657852, pvalue=0.00011886505415244596)
peak1_pear:  PearsonRResult(statistic=0.2328813694478455, pvalue=0.05414374124138603)
peak1_IC_pear:  PearsonRResult(statistic=0.38477387117524536, pvalue=0.001096807309917273)
CCCT_pear:  PearsonRResult(statistic=0.27951794295035653, pvalue=0.02001833832264209)
CVCT_pear:  PearsonRResult(statistic=-0.09217559074412551, pvalue=0.4512781585411179)
V37_419_pear:  PearsonRResult(statistic=0.2978399671141844, pvalue=0.012935622974135818)
V38_419_pear:  PearsonRResult(statistic=0.2978399671141844, pvalue=0.012935622974135818)
V37_41_pear:  PearsonRResult(statistic=0.1711844039495784, pvalue=0.15960774398275424)
V38_41_pear:  PearsonRResult(

In [2]:


Aki_IC = Akima1DInterpolator(vol42[p1],IC_s[p1])
IC_s2 = Aki_IC(vol42[p1])
IC_s3 = savgol_filter(IC_s2,int(len(vol42[p1])/2)*2-1,18)

n=1.5
IQR = np.percentile(IC_s3,75) - np.percentile(IC_s3,25)
IC_s3_1=IC_s3[IC_s3 < np.percentile(IC_s3,75)+n*IQR ]
IC_s3_1=IC_s3[IC_s3 > np.percentile(IC_s3,25)-n*IQR]
out_p = np.where(np.append(IC_s3 < np.percentile(IC_s3,25)-n*IQR,IC_s3 > np.percentile(IC_s3,75)+n*IQR))[0]
p1 = np.delete(p1,out_p)
k1 = get_IC_peaks(vol42[p1],IC_s3_1)


#outlier = Q1 - n*IQR 
# k1=k1[data['x'] > np.percentile(data['x'],25)-n*IQR]

fig5 = px.scatter(x = vol42[p1] ,y = IC_s3_1)
fig5.add_scatter(x=vol42[p1],y=IC_s[p1])
p2 = p1[0:-2]
fig5.add_scatter(x=vol42[p2],y=k1*10)
fig5.show(config=plotly_config)

NameError: name 'vol42' is not defined

In [44]:
np.array([])[0]

IndexError: index 0 is out of bounds for axis 0 with size 0